In [52]:
#%%writefile Cointegration.py
#https://medium.com/@bart.chr/pairs-trading-for-algorithmic-trading-breakdown-d8b709f59372
#https://github.com/aconstandinou/mean-reversion

import statsmodels.api as sm
import statsmodels.tsa.stattools as ts 
from statsmodels.tsa.stattools import coint
import numpy as np
import pandas as pd
import statistics

import matplotlib.pyplot as plt

# Functions/Methods

In [53]:
"""
Augmented Dickey–Fuller (ADF) unit root test; p-value < .05
"""
class DickeyFuller(object):
    def __init__(self, significance=.05):
        self.significance_level = significance
        self.p_value = None
        self.perc_stat = None
        self.is_stationary = None
        
    def check(self, time_series):
        model = ts.adfuller(time_series, 1)
        self.p_value = model[1]
        self.perc_stat = model[0]
        
        self.is_stationary = False
        if (self.p_value < self.significance_level):
            self.is_stationary = True
        return self.is_stationary;

"""
Half Life test from the Ornstein-Uhlenbeck process 
"""
class HalfLife(object):
    def __init__(self):
        self.half_life = None

    def check(self, time_series):
        lag = np.roll(time_series, 1)
        lag[0] = 0
        ret = time_series - lag
        ret[0] = 0

        # adds intercept terms to X variable for regression
        lag2 = sm.add_constant(lag)
        res = sm.OLS(ret, lag2).fit()
        self.half_life = int(round(-np.log(2) / res.params[1],0))

        if self.half_life <= 0:
            self.half_life = 1
        return self.half_life

"""
If Hurst Exponent is under the 0.5 value of a random walk, then the series is mean reverting
"""
class HurstExponent():
    def __init__(self):
        self.h_min = 0.0
        self.h_max = 0.4
        self.look_back = 126
        #https://robotwealth.com/demystifying-the-hurst-exponent-part-1/
        self.lag_max = 20#era 100
        self.h_value = None
    
    def check(self, time_series):
        lags = range(2, self.lag_max)

        tau = [np.sqrt(np.std(np.subtract(time_series[lag:], time_series[:-lag]))) for lag in lags]
        poly = np.polyfit(np.log(lags), np.log(tau), 1)

        self.h_value = poly[0]*2.0 
        return self.h_value

def model_ols(y, x):
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    return model

# beta/coeficiente angular
def beta(y, x):
    model = model_ols(y, x)
    return model.params[1]

# check cointegrated pairs from dataframe
def find_cointegrated_pairs(data, num_pairs=0):
    adf = DickeyFuller()
    rows = []
    isBreak = False
    index=-1
    
    for y_symbol in data.columns:
        index = index + 1
        for x_symbol in data.columns[index+1:data.shape[1]]:#for x_symbol in data.columns:
            if (y_symbol == x_symbol):
                continue
            
            model = model_ols(data[y_symbol], data[x_symbol])
            adf.check(model.resid)
            beta = model.params[1]
            #if (adf.is_stationary):
            rows.append([y_symbol,x_symbol,adf.p_value, adf.perc_stat, beta])
                    
            # break for two
            isBreak = (num_pairs > 0 and len(rows) >= num_pairs)
            if (isBreak == True): break
        
        # break for one
        if (isBreak == True): break

    df_pairs = pd.DataFrame(rows, columns=['Dependente', 'Independente', 'pValue', 'ADFStatistic', 'Beta'])
    return df_pairs

def apply_periods(data, pairs):
    pairs['Period'] = 0
    pairs['PeriodStr'] = ''    
    for i, row in pairs.iterrows():
        y = data[row['Dependente']]
        x = data[row['Independente']]
        analysis = analysis_by_periods(y, x)
        stationary = analysis.loc[(analysis['Stationary'])]

        des = ''
        for j, row in stationary.iterrows():
            if (des!=''):
                des=des+','
            des=des+str(row['Period'])

        pairs['Period'].iloc[i] = stationary.shape[0]
        pairs['PeriodStr'].iloc[i] = des
        
def analysis_by_periods(y, x):
    rows=[]
    n = len(y)
    adf = DickeyFuller()
    
    for period in [100, 120, 140, 160, 180, 200, 220, 240, 250]:
        pos = n-period
        y_values = y.iloc[pos:]
        x_values = x.iloc[pos:]
        
        coin = cointegration(y_values, x_values, 0)
        half_life = check_halflife(y_values, x_values)
        hurst = check_hurst(y_values, x_values)
        corr = corr_pearson(y_values, x_values)
            
        rows.append([period, coin[0], coin[1], coin[1], coin[2], half_life, hurst, corr])
        
    analysis = pd.DataFrame(rows, columns=['Period', 'Stationary', 'pValue', 'ADFStatistic', 'Beta', 'HalfLife', 'Hurst', 'Corr'])
    return analysis

def cointegration(y, x, period = 0):
    adf = DickeyFuller()
    n = len(y)
    if (period > 0):
        pos = n-period
        y = y.iloc[pos:]
        x = x.iloc[pos:]
    else:
        period = n
        
    model = model_ols(y, x)
    adf.check(model.resid)
    return [adf.is_stationary, adf.p_value, adf.perc_stat, model.params[1], period]

def apply_halflife(data, pairs):
    pairs['HalfLife'] = 0
    
    for i, row in pairs.iterrows():
        y = data[row['Dependente']]
        x = data[row['Independente']]
        
        value = check_halflife(y, x)
        pairs['HalfLife'].iloc[i]=value

def check_halflife(y, x):
    halflile = HalfLife()
    model = model_ols(y, x)
    return halflile.check(model.resid)

def apply_hurst(data, pairs):
    pairs['Hurst'] = 0
    
    for i, row in pairs.iterrows():
        y = data[row['Dependente']]
        x = data[row['Independente']]
        
        value = check_hurst(y, x)
        pairs['Hurst'].iloc[i]= value

def check_hurst(y, x):
    hurst = HurstExponent()
    model = model_ols(y, x)
    return hurst.check(model.resid.values)

# 0.9 para mais ou para menos indica uma correlação muito forte.
# 0.7 a 0.9 positivo ou negativo indica uma correlação forte.percorre
# 0.5 a 0.7 positivo ou negativo indica uma correlação moderada.
# 0.3 a 0.5 positivo ou negativo indica uma correlação fraca.
# 0 a 0.3 positivo ou negativo indica uma correlação desprezível.'''
def corr_pearson(y, x):
    y_avg, x_avg = np.average(y), np.average(x)
    y_stdev, x_stdev = np.std(y), np.std(x)
    n = len(y)
    denominator = y_stdev * x_stdev * n
    numerator = np.sum(np.multiply(y-y_avg, x-x_avg))
    p_coef = numerator/denominator
    return p_coef

def apply_corr(data, pairs):
    pairs['Corr'] = 0
    
    for i, row in pairs.iterrows():
        y = data[row['Dependente']]
        x = data[row['Independente']]
        
        corr = corr_pearson(y, x)
        pairs['Corr'].iloc[i] = corr

def signal(y, x):
    model = model_ols(y, x)
    std = statistics.stdev(model.resid)
    resi_curr = model.resid.iloc[-1]
    zscore_up = 2*std
    zscore_down = -2*std
    zcurrent = 0
    desc = ''
    
    # >0; resíduo acima da linha 0
    if(resi_curr > 0):
        desc = 'SL'
        zcurrent = zscore_up
    else:
        desc = 'LS'
        zcurrent = zscore_down
    
    percent = (abs(resi_curr)/abs(zcurrent))
    #1-descr
    #2-resíduo atual
    #3-percent distância da linha 0, quanto maior, melhor
    return [desc, resi_curr, percent]

def apply_signal(data, pairs):
    pairs['Signal'] = 0
    pairs['SignalStr'] = ''    
    
    for i, row in pairs.iterrows():
        y = data[row['Dependente']]
        x = data[row['Independente']]

        sig = signal(y, x)
        pairs['Signal'].iloc[i] = sig[2]
        pairs['SignalStr'].iloc[i] = sig[0]        

def check_periods(data, y_symbol, x_symbol, period):
    if (type(period) is int):
        return check_oneperiod(data, y_symbol, x_symbol, period)
    if (type(period) is list):
        rows=[]
        for p in period:
            res = check_oneperiod(data, y_symbol, x_symbol, p)
            rows.append([res[0], res[1]])
        return rows

def check_oneperiod(data, y_symbol, x_symbol, period):
    y = data[y_symbol]
    x = data[x_symbol]
    
    if(period > 0):
        pos = data.shape[0]-period
        y = y.iloc[pos:]
        x = x.iloc[pos:]
    
    adf = DickeyFuller()
    model = model_ols(y, x)
    adf.check(model.resid)
    beta = model.params[1]

    return [adf.p_value, adf.is_stationary];

def show(data, y_symbol, x_symbol, period=0):
    y = data[y_symbol]
    x = data[x_symbol]
    
    if(period > 0):
        pos = data.shape[0]-period
        y = y.iloc[pos:]
        x = x.iloc[pos:]
    
    model= model_ols(y, x)
    std = statistics.stdev(model.resid)
    entry_threshold = 2 # entrada em 2 desvio padrão

    #plt.figure(figsize=(15,6))
    plt.figure(figsize=(10,5))
    plt.plot(model.resid)
    plt.ylabel('Residual')
    plt.title(y_symbol + ' / ' + x_symbol)

    plt.axhline(0, color='black',label='mean',linestyle='--') # Add the mean of residual
    plt.axhline(entry_threshold*std, color='green', linestyle='--',label='trade')
    plt.axhline(-entry_threshold*std, color='green', linestyle='--')

    plt.legend()
    plt.show()

## Read CSV

In [57]:
# todas as ações
#df = pd.read_csv('datasets/data.csv')

# somentes as ações que fazem parte da carteira ibovespa
#df = pd.read_csv('datasets/data_cart.csv',index_col=0)#[['ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3']]

# Yahoo: somentes as ações que fazem parte da carteria ibovespa
df = pd.read_csv('datasets/data_yahoo.csv')

data = df[df.columns.difference(['Data', 'Date'])]
data.shape

(250, 75)

In [58]:
pairs = find_cointegrated_pairs(data, 0)
pairs.head(3)

MissingDataError: exog contains inf or nans

In [74]:
apply_halflife(data, pairs)
print('Half-Life applied successfully')

Half-Life applied successfully


In [75]:
apply_hurst(data, pairs)
print('Hurst applied successfully')

Hurst applied successfully


In [76]:
apply_corr(data, pairs)
print('Correlation applied successfully')

Correlation applied successfully


In [77]:
apply_signal(data, pairs)
print('Signal applied successfully')

Signal applied successfully


In [78]:
pairs.head(3)

,Dependente,Independente,pValue,ADFStatistic,Beta,HalfLife,Hurst,Corr,Signal,SignalStr
0,AALR3,AAPL34,0.751632,-1.004467,-0.139510,25,0.630012,-0.510495,0.174045,SL
1,AALR3,ABCB4,0.012558,-3.355893,1.338227,4,0.229039,0.977998,0.072126,LS
2,AALR3,ABEV3,0.115942,-2.498079,1.781097,8,0.460375,0.927493,0.433493,LS


In [79]:
pairs.to_csv('datasets/cointegrated_pairs.csv', index=False)
print('Criado com sucesso!!!')

Criado com sucesso!!!


# Analysis

In [50]:
pairs = pd.read_csv('datasets/cointegrated_pairs.csv')
pairs.shape

(2485, 10)

In [51]:
df_hurst = pairs.loc[(pairs['Hurst'] > 0) & (pairs['Hurst'] < 0.4)]
df_hurst

,Dependente,Independente,pValue,ADFStatistic,Beta,HalfLife,Hurst,Corr,Signal,SignalStr
2,ABEV3,BBAS3,0.026171,-3.105199,0.264844,14,0.394994,0.876870,0.019044,SL
5,ABEV3,BBSE3,0.259151,-2.064202,0.455859,28,0.317025,0.740923,0.358532,LS
10,ABEV3,BRFS3,0.227100,-2.144243,0.269852,27,0.379715,0.772527,0.402355,LS
12,ABEV3,BRML3,0.479478,-1.608190,0.659995,38,0.351098,0.705307,0.356065,LS
14,ABEV3,CCRO3,0.636006,-1.285001,0.556144,63,0.322082,0.531443,0.650193,LS
...,...,...,...,...,...,...,...,...,...,...
2463,TIMP3,WEGE3,0.159416,-2.339920,0.153076,15,0.320761,0.804171,0.635979,LS
2467,TOTS3,VIVT4,0.000306,-4.392164,3.090259,7,0.291917,0.837527,1.295110,LS
2475,USIM5,VALE3,0.330135,-1.904099,0.252504,29,0.366080,0.595611,0.966254,LS
2482,VIVT4,VVAR3,0.265442,-2.049162,0.887282,22,0.309864,0.700643,1.396879,LS


In [52]:
corr = df_hurst.loc[(df_hurst['Corr'] >= 0.70)]
corr

,Dependente,Independente,pValue,ADFStatistic,Beta,HalfLife,Hurst,Corr,Signal,SignalStr
2,ABEV3,BBAS3,0.026171,-3.105199,0.264844,14,0.394994,0.876870,0.019044,SL
5,ABEV3,BBSE3,0.259151,-2.064202,0.455859,28,0.317025,0.740923,0.358532,LS
10,ABEV3,BRFS3,0.227100,-2.144243,0.269852,27,0.379715,0.772527,0.402355,LS
12,ABEV3,BRML3,0.479478,-1.608190,0.659995,38,0.351098,0.705307,0.356065,LS
21,ABEV3,CVCB3,0.029495,-3.062018,0.130948,17,0.372962,0.854393,0.131519,LS
...,...,...,...,...,...,...,...,...,...,...
2455,TAEE11,VVAR3,0.027425,-3.088386,0.511639,10,0.320701,0.879774,1.269494,LS
2462,TIMP3,VVAR3,0.139840,-2.406753,0.441571,13,0.333959,0.853058,1.021864,LS
2463,TIMP3,WEGE3,0.159416,-2.339920,0.153076,15,0.320761,0.804171,0.635979,LS
2467,TOTS3,VIVT4,0.000306,-4.392164,3.090259,7,0.291917,0.837527,1.295110,LS


#### Pares sinalizando entrada

In [341]:
signal = corr.loc[(df_hurst['Signal'] >= 0.95)].copy()
signal.reset_index(drop=True, inplace=True)
apply_periods(data, signal)

In [356]:
# somente pares cointegrados em no mínimo 3 períodos
signal.loc[(signal['Period'] > 2)].sort_values(by=['HalfLife'], ascending=True)

,Dependente,Independente,pValue,ADFStatistic,Beta,HalfLife,Hurst,Corr,Signal,SignalStr,Period,PeriodStr
62,LREN3,MULT3,0.000068,-4.750407,1.512641,5,0.250833,0.971247,1.438070,SL,6,"140,160,180,220,240,250"
74,TOTS3,VIVT4,0.000125,-4.606812,3.066138,7,0.293995,0.841575,1.042910,LS,7,"140,160,180,200,220,240,250"
65,QUAL3,VIVT4,0.000774,-4.158656,1.836992,7,0.257253,0.902282,1.088393,SL,8,"120,140,160,180,200,220,240,250"
58,IGTA3,LREN3,0.000744,-4.168615,1.050753,7,0.297478,0.961280,1.379036,LS,7,"120,140,160,180,200,220,250"
14,BBAS3,EMBR3,0.004834,-3.652426,1.923078,8,0.380021,0.953200,0.997803,SL,4,"200,220,240,250"
47,FLRY3,VIVT4,0.001561,-3.972651,0.787924,9,0.336699,0.879504,1.338015,SL,6,"160,180,200,220,240,250"
70,SULA11,VIVT4,0.001682,-3.952340,2.185313,11,0.322983,0.867586,1.101672,SL,5,"180,200,220,240,250"
57,HYPE3,VIVT4,0.006323,-3.571833,0.619796,11,0.356265,0.770722,1.234197,SL,8,"120,140,160,180,200,220,240,250"
41,ECOR3,VIVT4,0.001841,-3.927544,0.570259,11,0.280518,0.883544,1.553927,SL,6,"160,180,200,220,240,250"
0,AZUL4,BBSE3,0.037784,-2.970082,3.152524,13,0.354216,0.919744,1.167227,LS,9,"100,120,140,160,180,200,220,240,250"
